In [28]:
from quantinuum_wrapper import QuantinuumWrapper
from pytket import Circuit
from qiskit_aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from sklearn.cluster import SpectralClustering
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import Sampler
import matplotlib.pyplot as plt
import numpy as np

def quantum_asset_pruning(correlation_matrix, esg_scores, num_clusters=10):
    """
    Quantum kernel-based clustering for asset pruning before optimization.
    Uses Qiskit QuantumKernel + Spectral clustering.
    """
    n = correlation_matrix.shape[0]
    backend = AerSimulation.get_backend('statevector_simulator')
    qi = QuantumInstance(backend)
    qk = QuantumKernel(quantum_instance=qi)

    # Compute pairwise quantum kernel matrix based on correlation and ESG data
    kernel_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            # Combine correlation and ESG into joint state (normalized)
            vec_i = np.hstack((correlation_matrix[i], esg_scores[i]))
            vec_j = np.hstack((correlation_matrix[j], esg_scores[j]))
            vec_i /= np.linalg.norm(vec_i) if np.linalg.norm(vec_i) > 0 else 1
            vec_j /= np.linalg.norm(vec_j) if np.linalg.norm(vec_j) > 0 else 1
            kernel_matrix[i, j] = qk.evaluate(x_vec=vec_i.reshape(1, -1), y_vec=vec_j.reshape(1, -1))

    # Apply quantum-inspired spectral clustering
    clustering = SpectralClustering(n_clusters=num_clusters, affinity='precomputed', random_state=42)
    labels = clustering.fit_predict(kernel_matrix)

    # Select one representative (highest ESG) per cluster
    selected_assets = []
    for k in range(num_clusters):
        cluster_indices = np.where(labels == k)[0]
        if len(cluster_indices) > 0:
            best_idx = cluster_indices[np.argmax(esg_scores[cluster_indices])]
            selected_assets.append(best_idx)
    return np.array(selected_assets)


def quantum_feature_selection(assets, correlation_matrix, esg_scores, target_clusters=10):
    """
    Integrates quantum kernel pruning into the classical optimization pipeline.
    """
    print(f"\n⚛️ Running Quantum Feature Selection on {len(assets)} assets...")
    selected_idx = quantum_asset_pruning(correlation_matrix, esg_scores, num_clusters=target_clusters)
    reduced_assets = np.array(assets)[selected_idx]
    print(f"✅ Reduced asset universe to {len(reduced_assets)} representative assets.\n")
    return reduced_assets


def run(input_data, solver_params, extra_arguments):
    """
    Integrated classical + quantum feature selection + quantum optimization pipeline.
    """
    from utils import Json_parse
    price_df, esg_s, asset_arr = Json_parse.json_parse(input_data)

    from utils import Pre_processing
    mu, Sigma = Pre_processing.compute_mu_sigma(price_df)
    plt.imshow(Sigma, interpolation="nearest"); plt.title("Covariance Matrix Σ"); plt.show()

    # -----------------------------------------
    # 🔹 STEP 1: Quantum Feature Selection
    # -----------------------------------------
    esg_arr = esg_s.loc[asset_arr].values
    selected_assets = quantum_feature_selection(asset_arr, Sigma, esg_arr, target_clusters=10)

    # Filter everything downstream
    Sigma_reduced = Sigma[np.ix_(selected_assets, selected_assets)]
    mu_reduced = mu[selected_assets]
    esg_reduced = esg_arr[selected_assets]

    # -----------------------------------------
    # 🔹 STEP 2: Classical Portfolio Optimization
    # -----------------------------------------
    from utils import run_portfolio_optimization_suite
    config_example = {
        'compute_inputs': {'shrinkage': True},
        'scalarized_qp': {
            'lamb_grid': [0.5, 1.0, 2.0],
            'eta_grid': [0.0, 0.2, 0.5],
            'lb': 0.0, 'ub': 0.2, 'allow_short': False
        },
        'cvar': {
            'alpha': 0.90, 'lamb': 0.5, 'eta': 0.3, 'use_mu_in_obj': True
        }
    }
    results_dict = run_portfolio_optimization_suite(
        price_df[selected_assets], mu_reduced, Sigma_reduced,
        selected_assets, pd.Series(esg_reduced, index=selected_assets),
        config_example
    )
    print("\n--- Classical Optimization Results Summary ---")
    print(results_dict.keys())

    # -----------------------------------------
    # 🔹 STEP 3: Quantum QAOA Refinement (Optional)
    # -----------------------------------------
    from utils import PortfolioOptimization, print_result
    q = 1  
    num_assets = len(mu_reduced)
    budget = num_assets // 2 
    portfolio = PortfolioOptimization(
        expected_returns=mu_reduced, covariances=Sigma_reduced, risk_factor=q,
        budget=budget, esg_scores=pd.Series(esg_reduced),
        roi_factor=1, esg_factor=0.01
    )
    qp = portfolio.to_quadratic_program()

    # Quantinuum backend
    provider = QuantinuumWrapper()
    backend = provider.get_backend('H1-1E')  # emulator first

    cobyla = COBYLA()
    cobyla.set_options(maxiter=250)
    qaoa_mes = QAOA(sampler=Sampler(backend=backend), optimizer=cobyla, reps=3)
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    result = qaoa.solve(qp)
    print_result(result)

    return {"result": "QAOA optimization complete."}


ImportError: cannot import name 'QuantumInstance' from 'qiskit.utils' (/opt/conda/lib/python3.12/site-packages/qiskit/utils/__init__.py)